### BRIEF (Binary Robust Independent Elementary Features)
SIFT는 128 개의 부동 소수점 숫자가있는 특성 설명자를 사용합니다. 이러한 기능의 수천을 고려하십시오. 매칭에 많은 시간과 메모리가 필요합니다. 더 빨리 압축 할 수 있습니다. 하지만 여전히 먼저 계산해야합니다. 메모리가 적고 더 빠른 매칭, 더 높은 인식률로 이진 디스크립터를 찾을 수있는 지름길을 제공하는 BRIEF가있다.

#### 목표
In this chapter

* We will see the basics of BRIEF algorithm

#### 이론
우리는 SIFT가 디스크립터에 128-dim vector를 사용한다는 것을 알고있다. 부동 소수점 숫자를 사용하기 때문에 기본적으로 512 바이트를 사용합니다. 유사하게 SURF는 최소 256 바이트 (64-dim)를 취합니다. 수천 개의 피쳐를위한 그러한 벡터를 생성하는 것은 많은 메모리를 필요로하는데, 이는 특히 임베디드 시스템을위한 resouce-constraint 애플리케이션에는 적합하지 않습니다. 메모리가 클수록 일치하는 데 걸리는 시간이 길어집니다.

그러나 이러한 모든 차원은 실제 일치에 필요하지 않을 수 있습니다. PCA, LDA 등과 같은 여러 가지 방법을 사용하여 압축 할 수 있습니다. LSH (Locality Sensitive Hashing)를 사용하는 해싱과 같은 다른 방법도 부동 소수점 숫자의 SIFT 설명자를 이진 문자열로 변환하는 데 사용됩니다. 이진 문자열은 해밍 거리를 사용하여 지형지 물을 일치시키는 데 사용됩니다. 해밍 거리를 찾는 것은 XOR과 비트 수를 적용하는 것 뿐이므로 SSE 명령을 사용하는 최신 CPU에서는 매우 빠르므로 더 빠른 속도 향상을 제공합니다. 하지만 여기서 우리는 디스크립터를 먼저 찾아야하며, 메모리 상에있는 우리의 초기 문제를 해결하지 못하는 해시를 적용 할 수 있습니다.

이 순간에 간단한 그림이 나온다. 설명자를 찾지 않고 바로 이진 문자열을 찾을 수있는 바로 가기를 제공합니다. 이는 매끄러운 이미지 패치를 취하여 고유 한 방식으로 \\(n_d (x, y)\\) 위치 쌍을 선택합니다 (종이에서 설명 함). 그런 다음 일부 픽셀 강도 비교가 이러한 위치 쌍에서 수행됩니다. 예를 들어, 첫 번째 위치 쌍을 \\(p\\)와 \\(q\\)라고 합시다. \\(I(p) < I(q)\\)이면 결과는 1이고 그렇지 않으면 0입니다. 이것은 \\(n_d\\) 차원 비트 스트링을 얻기 위해 모든 \\(nd\\) 위치 쌍에 적용됩니다.

이 번호는 128, 256 또는 512 일 수 있습니다. OpenCV는이 모든 것을 지원하지만 기본적으로 256입니다 (OpenCV는 바이트로 표시하므로 값은 16, 32 및 64가됩니다). 따라서 이것을 얻은 후에 Hamming Distance를 사용하여이 디스크립터와 일치시킬 수 있습니다.

한 가지 중요한 점은 기능 설명자인 BRIEF가 기능을 찾을 수있는 방법을 제공하지 않는다는 것입니다. 따라서 SIFT, SURF 등과 같은 다른 피쳐 감지기를 사용해야합니다.이 용지에서는 고속 감지기 인 CenSurE를 사용하는 것이 좋으며 BRIEF는 SURF 포인트보다 CenSurE 포인트가 약간 더 좋습니다.

간단히 말해서, BRIEF는 더 빠른 방법 특징 기술자 계산 및 매칭입니다. 또한 평면 회전이 큰 경우가 아니면 높은 인식률을 제공합니다.

#### BRIEF in OpenCV
아래 코드는 CenSurE 검출기를 사용하여 간단한 설명자를 계산하는 방법을 보여줍니다. (CenSurE 검출기는 OpenCV에서 STAR 검출기라고 부름) [opencv contrib](https://github.com/opencv/opencv_contrib) 이 필요함을주의 하십시오.)

In [1]:
import numpy as np
import cv2 as cv
from matplotlib import pyplot as plt

img = cv.imread('simple.jpg',0)

# Initiate FAST detector
star = cv.xfeatures2d.StarDetector_create()

# Initiate BRIEF extractor
brief = cv.xfeatures2d.BriefDescriptorExtractor_create()

# find the keypoints with STAR
kp = star.detect(img, None)

# compute the descriptors with BRIEF
kp, des = brief.compute(img, kp)

img3 = cv.drawKeypoints(img, kp, None, color=(0,0,255))


print( brief.descriptorSize() )
print( des.shape )

32
(33, 32)


The function brief.getDescriptorSize() gives the \\(n_d\\) size used in bytes. By default it is 32. Next one is matching, which will be done in another chapter.

#### Additional Resources
Michael Calonder, Vincent Lepetit, Christoph Strecha, and Pascal Fua, "BRIEF: Binary Robust Independent Elementary Features", 11th European Conference on Computer Vision (ECCV), Heraklion, Crete. LNCS Springer, September 2010.
[LSH (Locality Sensitive Hashing)](https://en.wikipedia.org/wiki/Locality-sensitive_hashing) at wikipedia.